# Project Fundamentals

In [7]:
import cv2
import mediapipe as mp

In [11]:
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode = False,
    max_num_hands = 2,
    model_complexity = 1,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
)

while cap.isOpened():
    success, img = cap.read()

    if not success: break

    img = cv2.flip(img, 1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)


    if results.multi_hand_landmarks:
        for hand_landmarks, hand_info in zip(results.multi_hand_landmarks, results.multi_handedness):
            hand_type = hand_info.classification[0].label
            thumb_tip = hand_landmarks.landmark[4]
            thumb_ip = hand_landmarks.landmark[3]

            thumb_status = 'Closed'

            if hand_type == 'Right':
                if thumb_tip.x < thumb_ip.x:
                    thumb_status = 'Open'
            else:
                if thumb_tip.x > thumb_ip.x:
                    thumb_status = 'Open'


            h,w,c = img.shape
            cx, cy = int(thumb_tip.x*w), int(thumb_tip.y*h)

            color = (0,255,0) if thumb_status=='Open' else (255,0,0)

            cv2.circle(img, (cx, cy), 15, color, cv2.FILLED)
            cv2.putText(img, f'{hand_type}, Thumb:{thumb_status}', (cx-50, cy-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 2)

            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Thumb Tracker", img)

    if cv2.waitKey(5) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import HandTrackingModule as htm
import math

# 1. Setup
cap = cv2.VideoCapture(0)
detector = htm.handDetector(detectionCon=0.75) # Higher confidence for precision

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)

    # 2. Find Hands
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        # Define the 3 points for the thumb angle
        # Point 2 (MCP), Point 3 (IP), Point 4 (Tip)
        # lmList format is [id, x, y]
        p1 = (lmList[2][1], lmList[2][2]) # MCP
        p2 = (lmList[3][1], lmList[3][2]) # IP (Vertex)
        p3 = (lmList[4][1], lmList[4][2]) # Tip

        # 3. Calculate Angle
        angle = htm.calculate_angle(p1, p2, p3)
        
        # 4. Determine State based on Angle
        # A straight thumb is ~180. A bent thumb is usually < 160.
        if angle > 160:
            thumb_state = "Straight"
            color = (0, 255, 0) # Green
        else:
            thumb_state = "Bent"
            color = (0, 0, 255) # Red

        # 5. Visual Feedback
        # Draw lines connecting the 3 points
        cv2.line(img, p1, p2, (255, 255, 255), 3)
        cv2.line(img, p2, p3, (255, 255, 255), 3)
        
        # Draw circles
        cv2.circle(img, p1, 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, p2, 10, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, p3, 10, (0, 0, 255), cv2.FILLED)
        
        # Display the Angle value and State
        cv2.putText(img, str(int(angle)), (p2[0] - 20, p2[1] + 50), 
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
        
        cv2.putText(img, thumb_state, (50, 50), 
                    cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

    cv2.imshow("Advanced Thumb Tracker", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()